# Assignment 1.3: Naive word2vec (40 points)

This task can be formulated very simply. Follow this [paper](https://arxiv.org/pdf/1411.2738.pdf) and implement word2vec like a two-layer neural network with matrices $W$ and $W'$. One matrix projects words to low-dimensional 'hidden' space and the other - back to high-dimensional vocabulary space.

![word2vec](https://i.stack.imgur.com/6eVXZ.jpg)

You can use TensorFlow/PyTorch and code from your previous task.

## Results of this task: (30 points)
 * trained word vectors (mention somewhere, how long it took to train)
 * plotted loss (so we can see that it has converged)
 * function to map token to corresponding word vector
 * beautiful visualizations (PCE, T-SNE), you can use TensorBoard and play with your vectors in 3D (don't forget to add screenshots to the task)

## Extra questions: (10 points)
 * Intrinsic evaluation: you can find datasets [here](http://download.tensorflow.org/data/questions-words.txt)
 * Extrinsic evaluation: you can use [these](https://medium.com/@dataturks/rare-text-classification-open-datasets-9d340c8c508e)

Also, you can find any other datasets for quantitative evaluation.

Again. It is **highly recommended** to read this [paper](https://arxiv.org/pdf/1411.2738.pdf)

Example of visualization in tensorboard:
https://projector.tensorflow.org

Example of 2D visualisation:

![2dword2vec](https://www.tensorflow.org/images/tsne.png)

In [1]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim

from pathlib import Path
from pprint import pprint

UNK_TOKEN = '<UNK>'

np.random.seed(4242)
random.seed(4242)

In [2]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

device

device(type='cuda')

In [3]:
from collections import Counter


class CBOWBatcher:
    THRESHOLD = 5
    def __init__(self, dataset, window_size=2, threshold=THRESHOLD):
        self.window_size = window_size
        self.threshold = threshold
        c = Counter(dataset)
        # all the words we have plus <UNK> token for rare words
        unique = {w for w in dataset if c[w] > self.threshold}
        self.word2ind = {w: i for i, w in enumerate(unique)}
        self.word2ind[UNK_TOKEN] = len(self.word2ind)
        self.ind2word = {i: w for w, i in self.word2ind.items()}
        # We need to store only the numbers of the words here, as we have their numbers already
        # we create a padded array for tokens to process all the words from corpus
        # remove all the uncommon words here
        self.tokens = ([self.word2ind[UNK_TOKEN]] * window_size) +\
            [self.word2ind.get(w, self.word2ind[UNK_TOKEN]) for w in dataset] +\
            ([self.word2ind[UNK_TOKEN]] * window_size)
        self.vocab_size = len(set(self.tokens))
        assert self.vocab_size == len(self.word2ind)
        assert all(t < self.vocab_size for t in self.tokens)
        pprint(f'Corpus size: {len(dataset)}')
        pprint(f'Actual count of words used: {self.vocab_size}')
        pprint(f'{len(dataset)} words in dataset tokenized to {len(self.tokens)} tokens')

    def get_batch(self, batch_size=512):
        X = [None] * batch_size
        y = [None] * batch_size
        current = 0
        for start in np.random.permutation(range(len(self.tokens) - 2 * window_size)):
            center = start + window_size
            X[current] = [self.tokens[i]
                          for i in range(center - window_size, center + window_size + 1) if i != center]
            y[current] = self.tokens[center]
            current += 1
            if current == batch_size:
                # We need the generator, so only `yield ` is an option here
                yield torch.from_numpy(np.asarray(X)).to(device=device),\
                      torch.from_numpy(np.asarray(y)).to(device=device)
                # clean the buffer after we yielded it and we got back our process here
                X = [None] * batch_size
                y = [None] * batch_size
                current = 0
        if current:
            # if batch didn't get to the full size but the corpus ended
            yield torch.from_numpy(np.asarray(X[:current])).to(device=device),\
                  torch.from_numpy(np.asarray(y[:current])).to(device=device)         


In [4]:
test8_Data = Path.cwd() / 'text8'
with test8_Data.open() as f:
    # 1. simple cleaning: lowering all the words
    text8 = [a.lower() for line in f for a in line.split()]
    batcher = CBOWBatcher(text8, threshold=6)

'Corpus size: 17005207'
'Actual count of words used: 58113'
'17005207 words in dataset tokenized to 17005211 tokens'


In [5]:
class CBOWW2V(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size=256, window=2):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embedding_size * window * 2)
        self.W = nn.Linear(embedding_size * window * 2, hidden_size)
        nn.init.xavier_normal_(self.W.weight)
        self.relu = nn.ReLU(inplace=False)
        self.W1 = nn.Linear(hidden_size, vocab_size)
        nn.init.xavier_normal_(self.W1.weight)

    def forward(self, x):
        # get the embedding by indices
        x = self.embed(x)
        # hidden linear layer
        x = self.relu(self.W(x))
        # get the predictions
        x = self.W1(x)
        # we need only 1 word by the given ones
        # here we got 4 options, so let's average them
        return x.mean(dim=1)

    def get_word_emdedding(self, word):
        word = torch.LongTensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)


def test_CBOWW2V_shapes():
    window_size = 2
    batch_size = 64
    vocab_size = 50
    x = torch.zeros((batch_size, window_size * 2), dtype=torch.long)
    model = CBOWW2V(vocab_size, 42)
    scores = model(x)
    assert scores.size() == torch.Size([batch_size, vocab_size]), scores.size()


test_CBOWW2V_shapes()


In [10]:
from torch.utils.tensorboard import SummaryWriter


EACH_PRINT = 100
def train_model(model, optimizer, epochs=1):
    loss = nn.CrossEntropyLoss()
    total_loss = 0
    for e in range(epochs):
        with SummaryWriter() as writer:
            for t, (x, y) in enumerate(batcher.get_batch()):
                model.train()
                x = x.to(device=device, dtype=torch.long)
                y = y.to(device=device, dtype=torch.long)
                for xx in x:
                    if not all(xxx.item() <= batcher.vocab_size for xxx in xx):
                        pprint(batcher.tokens)
                        pprint(x)
                        break

                scores = model(x)
                check = loss(scores, y)

                optimizer.zero_grad()
                check.backward()
                optimizer.step()
                total_loss += check.data
                average_loss = float(total_loss / (len(x) * (t + 1)))
                writer.add_scalar('Loss/train', average_loss, t)
                if not t % EACH_PRINT:
                    pprint(f'Iteration {t}, loss = {average_loss:.4f}')


In [ ]:
pprint(device)

learning_rate = 1e-2
embedding_size = 222
window_size = 2
model = CBOWW2V(batcher.vocab_size, embedding_size)
model = model.to(device=device)
optimizer = optim.ASGD(model.parameters(), lr=learning_rate)

train_model(model, optimizer)

device(type='cuda')
'Iteration 0, loss = 0.0214'
